In [2]:
from gutenberg.cleanup import strip_headers
import requests
# http://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

def load_etext(num):
  url ='http://www.gutenberg.org/files/{0}/{0}-0.txt'.format(num)
  data= requests.get(url)
  return data.text

texts =[]
text_nums=[1342,76,11,2701,98,2591]
text_names =['Pride and Prejudice','Adventures of Huckleberry Finn',"Alice’s Adventures in Wonderland",
            "Moby Dick", "A Tale of Two Cities","Grimms’ Fairy Tales"]
for text_num in text_nums:
    text = strip_headers(load_etext(text_num)).strip()
    texts.append(text)

//anaconda/lib/python2.7/site-packages/cffi/model.py:532: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


# Preprocess Text

- Lowercase
- Strip Tags
- Strip Punctuation
- Strip Multiple Whitespaces
- Strip Numeric
- Remove Stopwords
- Strip Shortwords
- Stem Text


In [3]:
from gensim.parsing.preprocessing import preprocess_string,preprocess_documents
from gensim import corpora,models, similarities


In [4]:
processed_docs = preprocess_documents(texts)

In [5]:
dictionary = corpora.Dictionary(processed_docs)
dictionary.save('gutenberg.dict')

In [6]:
print(dictionary)

Dictionary(17922 unique tokens: [u'\u2014\u2018the', u'\u201cbroke', u'\u2018most', u'woodi', u'beauvai']...)


In [7]:
corpus = [dictionary.doc2bow(text) for text in processed_docs]
corpora.MmCorpus.serialize('gutenberg.mm', corpus) 

# Model

In [14]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=500)
lsi.save("gutenberg.model")
index = similarities.MatrixSimilarity(lsi[corpus])
# similarities.Similarity
index.save('gutenberg.index')

6

In [16]:
lsi.show_topics()

[(0,
  u'0.313*"said" + 0.242*"whale" + 0.188*"like" + 0.180*"time" + 0.167*"look" + 0.157*"man" + 0.149*"come" + 0.141*"littl" + 0.137*"old" + 0.128*"head"'),
 (1,
  u'0.520*"whale" + -0.343*"said" + 0.182*"ship" + 0.174*"sea" + 0.148*"ahab" + 0.141*"boat" + -0.122*"got" + -0.121*"went" + 0.107*"captain" + -0.105*"sai"'),
 (2,
  u'0.308*"sai" + 0.275*"got" + -0.224*"elizabeth" + 0.170*"jim" + 0.167*"didn\u2019t" + 0.146*"come" + 0.143*"ain\u2019t" + 0.142*"warn\u2019t" + -0.141*"darci" + 0.136*"don\u2019t"'),
 (3,
  u'0.372*"said" + -0.271*"elizabeth" + 0.201*"came" + -0.174*"sai" + -0.170*"darci" + -0.148*"mr" + -0.138*"bennet" + -0.131*"jane" + 0.130*"went" + -0.128*"know"'),
 (4,
  u'0.254*"lorri" + 0.217*"defarg" + 0.193*"hand" + 0.174*"look" + 0.170*"prison" + -0.155*"said" + -0.150*"elizabeth" + 0.144*"face" + -0.142*"went" + 0.138*"madam"'),
 (5,
  u'-0.680*"alic" + -0.279*"said" + 0.124*"came" + 0.107*"man" + -0.100*"turtl" + -0.099*"hatter" + -0.098*"mock" + -0.097*"gryphon" 

# Query

In [10]:
def query(doc):
    vec_bow = dictionary.doc2bow(preprocess_string(doc))
    vec_lsi = lsi[vec_bow] 
    return vec_lsi


In [11]:
query_texts = ["""  In fact, the artist's design seemed this: a final theory of my own,
partly based upon the aggregated opinions of many aged persons with whom
I conversed upon the subject. The picture represents a Cape-Horner in a
great hurricane; the half-foundered ship weltering there with its three
dismantled masts alone visible; and an exasperated whale, purposing to
spring clean over the craft, is in the enormous act of impaling himself
upon the three mast-heads.

The opposite wall of this entry was hung all over with a heathenish
array of monstrous clubs and spears. Some were thickly set with
glittering teeth resembling ivory saws; others were tufted with knots of
human hair; and one was sickle-shaped, with a vast handle sweeping round
like the segment made in the new-mown grass by a long-armed mower. You
shuddered as you gazed, and wondered what monstrous cannibal and savage
could ever have gone a death-harvesting with such a hacking, horrifying
implement. Mixed with these were rusty old whaling lances and harpoons
all broken and deformed. Some were storied weapons. With this once long
lance, now wildly elbowed, fifty years ago did Nathan Swain kill fifteen
whales between a sunrise and a sunset. And that harpoon--so like a
corkscrew now--was flung in Javan seas, and run away with by a whale,
years afterwards slain off the Cape of Blanco. The original iron entered
nigh the tail, and, like a restless needle sojourning in the body of a
man, travelled full forty feet, and at last was found imbedded in the
hump.

Crossing this dusky entry, and on through yon low-arched way--cut
through what in old times must have been a great central chimney with
fireplaces all round--you enter the public room. A still duskier place
is this, with such low ponderous beams above, and such old wrinkled
planks beneath, that you would almost fancy you trod some old craft's
cockpits, especially of such a howling night, when this corner-anchored
old ark rocked so furiously. On one side stood a long, low, shelf-like
table covered with cracked glass cases, filled with dusty rarities
gathered from this wide world's remotest nooks. Projecting from the
further angle of the room stands a dark-looking den--the bar--a rude
attempt at a right whale's head. Be that how it may, there stands the
vast arched bone of the whale's jaw, so wide, a coach might almost drive
beneath it. Within are shabby shelves, ranged round with old decanters,
bottles, flasks; and in those jaws of swift destruction, like another
cursed Jonah (by which name indeed they called him), bustles a little
withered old man, who, for their money, dearly sells the sailors
deliriums and death.

Abominable are the tumblers into which he pours his poison. Though
true cylinders without--within, the villanous green goggling glasses
deceitfully tapered downwards to a cheating bottom. Parallel meridians
rudely pecked into the glass, surround these footpads' goblets. Fill to
THIS mark, and your charge is but a penny; to THIS a penny more; and so
on to the full glass--the Cape Horn measure, which you may gulp down for
a shilling.

Upon entering the place I found a number of young seamen gathered about
a table, examining by a dim light divers specimens of SKRIMSHANDER. I
sought the landlord, and telling him I desired to be accommodated with a
room, received for answer that his house was full--not a bed unoccupied.
"But avast," he added, tapping his forehead, "you haint no objections
to sharing a harpooneer's blanket, have ye? I s'pose you are goin'
a-whalin', so you'd better get used to that sort of thing."

I told him that I never liked to sleep two in a bed; that if I should
ever do so, it would depend upon who the harpooneer might be, and
that if he (the landlord) really had no other place for me, and the
harpooneer was not decidedly objectionable, why rather than wander
further about a strange town on so bitter a night, I would put up with
the half of any decent man's blanket.

"I thought so. All right; take a seat. Supper?--you want supper?
Supper'll be ready directly."""""]
query_lsis = map(query,query_texts)

In [12]:
query_lsis[0]

[(0, 9.9644082839770203),
 (1, 6.0384039006323773),
 (2, 0.56334741419701762),
 (3, -0.14311474819062925),
 (4, 0.12710067100587508),
 (5, 0.71845019606324145)]

In [13]:
sims = index[query_lsis[0]]

In [14]:
print(list(enumerate(sims)))

[(0, 0.34231883), (1, 0.47632757), (2, 0.29236665), (3, 0.99579275), (4, 0.55444098), (5, 0.42607823)]


In [15]:
sorted(zip(text_names,sims),key=lambda x: x[1],reverse=True)

[('Moby Dick', 0.99579275),
 ('A Tale of Two Cities', 0.55444098),
 ('Adventures of Huckleberry Finn', 0.47632757),
 ('Grimms\xe2\x80\x99 Fairy Tales', 0.42607823),
 ('Pride and Prejudice', 0.34231883),
 ('Alice\xe2\x80\x99s Adventures in Wonderland', 0.29236665)]

# Moby Dick Matches Moby Dick